In [ ]:
!git clone https://github.com/iglu-contest/iglu
!apt-get -qq install openjdk-8-jdk xvfb > /dev/null
!update-alternatives --config java
!sudo add-apt-repository ppa:openjdk-r/ppa
!sudo apt-get update
!sudo apt-get install openjdk-8-jdk
!sudo apt-get install xvfb
!pip uninstall -y iglu && pip install git+https://github.com/iglu-contest/iglu.git
!pip install gym==0.18.3

In [1]:
import torch 
torch.__version__

'1.9.0+cu102'

In [ ]:
# exec this cell ONLY in colab
!wget -q https://raw.githubusercontent.com/iglu-contest/tutorials/main/env/colab_setup.sh -O - | sh > /dev/null 2>&1
!pip install -q pyvirtualdisplay
from pyvirtualdisplay import Display
disp = Display(backend="xvnc", size=(800, 600))
disp.start();
# for local notebooks instead launch jupyter as: xvfb-run -s "-screen 0 640x480x24" jupyter ...

In [1]:
import iglu
import gym
from iglu.tasks import RandomTasks
from iglu.tasks.task_set import TaskSet

#env = gym.make('IGLUSilentBuilder-v0')
#obs = env.reset()

/root/miniconda/envs/py37/lib/python3.7/site-packages/gym/logger.py:30: UserWarning: WARN: Box bound precision lowered by casting to float32
  warnings.warn(colorize('%s: %s'%('WARN', msg % args), 'yellow'))
/root/miniconda/envs/py37/lib/python3.7/site-packages/gym/logger.py:30: UserWarning: WARN: Box bound precision lowered by casting to float32
  warnings.warn(colorize('%s: %s'%('WARN', msg % args), 'yellow'))


In [2]:
!pip install sentence-transformers
#import nltk
from sentence_transformers import SentenceTransformer

#nltk.download('punkt')
bert_sentence = SentenceTransformer('all-distilroberta-v1')

     |████████████████████████████████| 78 kB 1.6 MB/s eta 0:00:011
     |████████████████████████████████| 3.1 MB 3.3 MB/s eta 0:00:01
     |████████████████████████████████| 3.3 MB 22.0 MB/s eta 0:00:01
     |████████████████████████████████| 23.2 MB 6.6 MB/s eta 0:00:01
     |████████████████████████████████| 1.5 MB 8.4 MB/s eta 0:00:01
     |████████████████████████████████| 1.2 MB 8.0 MB/s eta 0:00:01
     |████████████████████████████████| 59 kB 9.8 MB/s  eta 0:00:01
     |████████████████████████████████| 749 kB 13.9 MB/s eta 0:00:01
     |████████████████████████████████| 895 kB 13.1 MB/s eta 0:00:01
     |████████████████████████████████| 306 kB 16.1 MB/s eta 0:00:01
  Created wheel for sentence-transformers: filename=sentence_transformers-2.1.0-py3-none-any.whl size=120999 sha256=afe086c5c475ef55cde02f5ed36030a7f9ddf9d0677385622080f2663845ff99
  Stored in directory: /root/.cache/pip/wheels/90/f0/bb/ed1add84da70092ea526466eadc2bfb197c4bcb8d4fa5f7bad
Successfully built sentence

Downloading:   0%|          | 0.00/737 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/9.86k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/653 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/116 [00:00<?, ?B/s]

...

Downloading:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/333 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/13.1k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/798k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [4]:
import numpy as np

augmented_chats = np.load("data/augmented_chats.npy")
augmented_tasks = np.load("data/augmented_targets.npy")
augmented_target_names = np.load("data/augmented_target_name.npy")

In [10]:
targets = []
chats = []
for i in range(augmented_chats.shape[0]):
  targets.append(augmented_tasks[i])
  chats.append(bert_sentence.encode(augmented_chats[i]))

In [11]:
TaskSet(preset=['C32']).sample().target_grid[1]

array([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]], dtype=int32)

In [12]:
TaskSet(preset=['C32']).sample().chat

"<Builder> Mission has started.\n<Architect> hello I'm ready\n<Builder> same\n<Architect> I mean are you ready? lol mb\n<Architect> please don't move at the beginning of the missions\n<Builder> sorry\n<Architect> place 3 blue blocks horizontally anywhere you want\n<Builder> like that?\n<Architect> place 2 blue blocks on top of the left most blue block\n<Builder> relative to my position correct?\n<Architect> that's good"

In [13]:
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
import torch
from torch.nn.functional import one_hot

class TargetDataset(Dataset):
    def __init__(self, target_list, chat_list):
        self.target_list = target_list
        self.chat_list = chat_list

    def __len__(self):
        return len(self.target_list)

    def __getitem__(self, idx):
        target = self.target_list[idx]
        target_tensor_target = torch.tensor(target, dtype=torch.long)
        # target_tensor_input = one_hot(torch.tensor(target, dtype=torch.long), num_classes=7).permute(3, 0, 1, 2)
        chat_tensor = self.chat_list[idx]
        return chat_tensor, target_tensor_target
    
training_dataset = TargetDataset(targets, chats)

In [14]:
train_dataloader = DataLoader(training_dataset, batch_size=8, shuffle=True)

In [15]:
import torch
from torch import nn
from torch import optim
import numpy as np

device = torch.device('cuda')

class TargetDecoder(nn.Module):
    def __init__(self, features_dim=768):
        super(TargetDecoder, self).__init__()

        self.linear = nn.Sequential(nn.Linear(features_dim, 15680))

        self.cnn = nn.Sequential(
            nn.ConvTranspose3d(64, 64, kernel_size=3),
            nn.ReLU(),
            nn.ConvTranspose3d(64, 32, kernel_size=3, padding=1),
            nn.ReLU(), 
            nn.ConvTranspose3d(32, 7, kernel_size=3),
        )

    def forward(self, x):
        x = self.linear(x)
        x = x.reshape(x.shape[0], 64, 5, 7, 7)
        x = self.cnn(x)
        return x

    

target_decoder = TargetDecoder().to(device)
optimizer = optim.Adam(target_decoder.parameters(), lr=1e-3)
loss_function = nn.CrossEntropyLoss()

In [16]:
EPOCHS = 500
for epoch in range(EPOCHS):
  target_decoder.train()
  train_loss = []
  for target_tensor_input, target_tensor_target in train_dataloader:
    target_tensor_input = target_tensor_input.float().to(device)
    target_tensor_target = target_tensor_target.to(device)
    optimizer.zero_grad()
    predict = target_decoder(target_tensor_input)
    loss = loss_function(predict, target_tensor_target)
    train_loss.append(loss.item())
    loss.backward()
    optimizer.step()
  train_loss = np.array(train_loss).mean()
  print(f"epoch: {epoch} | loss: {train_loss}")

epoch: 0 | loss: 0.17597686499357224
epoch: 1 | loss: 0.09184326324351028
epoch: 2 | loss: 0.06355792848544628
epoch: 3 | loss: 0.046642813325470905
epoch: 4 | loss: 0.03567553744116584
epoch: 5 | loss: 0.030809795439598105
epoch: 6 | loss: 0.028476530174706496
epoch: 7 | loss: 0.02637559880635568
epoch: 8 | loss: 0.02513064543197469
epoch: 9 | loss: 0.024067364147856457
epoch: 10 | loss: 0.02312205295164545
epoch: 11 | loss: 0.02261968434681328
epoch: 12 | loss: 0.021684857567965315
epoch: 13 | loss: 0.02094266881399295
epoch: 14 | loss: 0.020177532457003072
epoch: 15 | loss: 0.01953671799831781
epoch: 16 | loss: 0.01940122362831477
epoch: 17 | loss: 0.018415056497735137
epoch: 18 | loss: 0.018226972001106467
epoch: 19 | loss: 0.017455583169576332
epoch: 20 | loss: 0.01746766170074626
epoch: 21 | loss: 0.016538566652768885
epoch: 22 | loss: 0.01620798382167222
epoch: 23 | loss: 0.015381728687665375
epoch: 24 | loss: 0.015547778604797372
epoch: 25 | loss: 0.014867684438678563
epoch: 26

KeyboardInterrupt: 

In [27]:
optimizer = optim.Adam(target_decoder.parameters(), lr=0.1e-5)
loss_function = nn.CrossEntropyLoss()

EPOCHS = 500
for epoch in range(EPOCHS):
  target_decoder.train()
  train_loss = []
  for target_tensor_input, target_tensor_target in train_dataloader:
    target_tensor_input = target_tensor_input.float().to(device)
    target_tensor_target = target_tensor_target.to(device)
    optimizer.zero_grad()
    predict = target_decoder(target_tensor_input)
    loss = loss_function(predict, target_tensor_target)
    train_loss.append(loss.item())
    loss.backward()
    optimizer.step()
  train_loss = np.array(train_loss).mean()
  print(f"epoch: {epoch} | loss: {train_loss}")

epoch: 0 | loss: 0.00023542717559543828
epoch: 1 | loss: 0.0002354207318455187
epoch: 2 | loss: 0.0002352376371716849
epoch: 3 | loss: 0.00023523254648319912
epoch: 4 | loss: 0.00023508454695145558
epoch: 5 | loss: 0.00023505252403008278
epoch: 6 | loss: 0.0002347181924306653
epoch: 7 | loss: 0.00023477412727427064
epoch: 8 | loss: 0.00023470913235955305
epoch: 9 | loss: 0.00023491479260610759
epoch: 10 | loss: 0.00023455589122753029
epoch: 11 | loss: 0.00023416805164318169
epoch: 12 | loss: 0.00023422336349132787
epoch: 13 | loss: 0.00023410076778036302
epoch: 14 | loss: 0.00023423585927272595
epoch: 15 | loss: 0.00023385543904665173
epoch: 16 | loss: 0.00023542086047007447
epoch: 17 | loss: 0.00023371252084906897
epoch: 18 | loss: 0.00023355064654956309
epoch: 19 | loss: 0.00023350801476259599
epoch: 20 | loss: 0.00023373320224094596
epoch: 21 | loss: 0.00023329254723675522
epoch: 22 | loss: 0.00023319843114295356
epoch: 23 | loss: 0.00023309962520564798
epoch: 24 | loss: 0.000239300

KeyboardInterrupt: 

In [28]:
for target_tensor_input, target_tensor_target in train_dataloader:
    target_tensor_input = target_tensor_input.float().to(device)
    target_tensor_target = target_tensor_target.to(device)
    predict = target_decoder(target_tensor_input)
    #print(predict.shape)
    predict = torch.argmax(nn.Softmax(dim=1)(predict).permute(0, 2, 3, 4, 1), dim=4)
    #print(predict.shape)
    #print(target_tensor_target.shape)
    print(torch.all(target_tensor_target == predict))
    #break

tensor(True, device='cuda:0')
tensor(False, device='cuda:0')
tensor(True, device='cuda:0')
tensor(True, device='cuda:0')
tensor(True, device='cuda:0')
tensor(True, device='cuda:0')
tensor(True, device='cuda:0')
tensor(True, device='cuda:0')
tensor(True, device='cuda:0')
tensor(True, device='cuda:0')
tensor(True, device='cuda:0')
tensor(True, device='cuda:0')
tensor(True, device='cuda:0')
tensor(True, device='cuda:0')
tensor(True, device='cuda:0')
tensor(True, device='cuda:0')
tensor(True, device='cuda:0')
tensor(True, device='cuda:0')
tensor(True, device='cuda:0')
tensor(True, device='cuda:0')
tensor(True, device='cuda:0')
tensor(False, device='cuda:0')
tensor(True, device='cuda:0')
tensor(True, device='cuda:0')
tensor(True, device='cuda:0')
tensor(False, device='cuda:0')
tensor(True, device='cuda:0')
tensor(True, device='cuda:0')
tensor(True, device='cuda:0')
tensor(True, device='cuda:0')
tensor(True, device='cuda:0')
tensor(True, device='cuda:0')
tensor(False, device='cuda:0')
tensor

tensor(True, device='cuda:0')
tensor(True, device='cuda:0')
tensor(True, device='cuda:0')
tensor(False, device='cuda:0')
tensor(True, device='cuda:0')
tensor(True, device='cuda:0')
tensor(False, device='cuda:0')
tensor(True, device='cuda:0')
tensor(True, device='cuda:0')
tensor(True, device='cuda:0')
tensor(True, device='cuda:0')
tensor(True, device='cuda:0')
tensor(True, device='cuda:0')
tensor(True, device='cuda:0')
tensor(False, device='cuda:0')
tensor(True, device='cuda:0')
tensor(True, device='cuda:0')
tensor(True, device='cuda:0')
tensor(False, device='cuda:0')
tensor(True, device='cuda:0')
tensor(True, device='cuda:0')
tensor(True, device='cuda:0')
tensor(True, device='cuda:0')
tensor(True, device='cuda:0')
tensor(True, device='cuda:0')
tensor(True, device='cuda:0')
tensor(True, device='cuda:0')
tensor(True, device='cuda:0')
tensor(True, device='cuda:0')
tensor(True, device='cuda:0')
tensor(True, device='cuda:0')
tensor(True, device='cuda:0')
tensor(True, device='cuda:0')
tensor

In [29]:
torch.save(target_decoder.state_dict(), 'target_decoder_aug.pth')